Try importing parquet file

In [5]:
import pandas as pd
#import click
from sqlalchemy import create_engine
from tqdm.auto import tqdm


parquet_path = "./green_tripdata_2025-11.parquet"

pg_user = 'postgres'
pg_pass = 'postgres'
pg_host = 'db'
pg_port = '5432'
pg_db = 'ny_taxi'

target_table = 'green_taxi_trips'

engine = create_engine(f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')

first = True

df_iter = pd.read_parquet(parquet_path, iterator=True, chunksize=100000)

for df_chunk in tqdm(df_iter):
    if first:
        df_chunk.head(0).to_sql(
            name=target_table,
            con=engine,
            if_exists='replace'
        )
        first = False

    df_chunk.to_sql(
        name=target_table,
        con=engine,
        if_exists='append'
    )


print(df.head())
print

ModuleNotFoundError: No module named 'sqlalchemy'